<a href="https://colab.research.google.com/github/siddharthkulgod/coursera-test/blob/main/Cascade_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os.path import exists, join, basename, splitext

# TODO: change URL to your fork of my repository if necessary.
git_repo_url = 'https://github.com/Tony607/mmdetection_instance_segmentation_demo'

%cd /content
project_name = os.path.abspath(splitext(basename(git_repo_url))[0])
mmdetection_dir = os.path.join(project_name, "mmdetection")
if not exists(project_name):
    # clone "depth 1" will only get the latest copy of the relevant files.
    !git clone -q --recurse-submodules --depth 1 $git_repo_url
    print("Update mmdetection repo")
    !cd {mmdetection_dir} && git checkout master && git pull
    # dependencies
    !pip install -q mmcv terminaltables
    # build
    !cd {mmdetection_dir} && python setup.py install
    !pip install -r {os.path.join(mmdetection_dir, "requirements.txt")}

/content
Update mmdetection repo
Previous HEAD position was ff284133 Remove redundant data transforms (#1522)
Switched to branch 'master'
Your branch is up to date with 'origin/master'.
Already up to date.
     |████████████████████████████████| 313 kB 16.2 MB/s 
     |████████████████████████████████| 185 kB 55.2 MB/s 
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running install
running bdist_egg
running egg_info
creating mmdet.egg-info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
writing manifest file 'mmdet.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mmdet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mmdet
copying mmdet/v

In [ ]:
!pip install mmcv-full

     |████████████████████████████████| 313 kB 15.1 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.13-cp37-cp37m-linux_x86_64.whl size=21813442 sha256=25bdca35bf30342c956ee98f46080eb76a142e80268e5211fc8cfff5355f9c0e
  Stored in directory: /root/.cache/pip/wheels/9d/13/23/7c772e752c5e05ef7fc7328dccefc152df8ab7bbf66b2ee59c
Successfully built mmcv-full


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect.
import os
from os.path import exists, join, basename, splitext

# TODO: change URL to your fork of my repository if necessary.
git_repo_url = 'https://github.com/Tony607/mmdetection_instance_segmentation_demo'

%cd /content
project_name = os.path.abspath(splitext(basename(git_repo_url))[0])
mmdetection_dir = os.path.join(project_name, "mmdetection")

/content


In [ ]:
import os
import sys
sys.path.append(mmdetection_dir)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

# You can add more model configs like below.
MODELS_CONFIG = {
    'mask_rcnn_r50_fpn_1x': {
        'config_file': 'configs/mask_rcnn_r50_fpn_1x.py'
    },
    'cascade_mask_rcnn_r50_fpn_1x': {
        'config_file': 'configs/cascade_mask_rcnn_r50_fpn_1x.py'
    },
    # 'cascade_rcnn_r50_fpn_1x': {
    #     'config_file': 'configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x.py',
    # }
    'cascade_rcnn_r50_fpn_1x': {
        'config_file': '/content/mmdetection_instance_segmentation_demo/mmdetection/configs/_base_/models/cascade_rcnn_r50_fpn.py'
    }
}

In [ ]:
# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'cascade_rcnn_r50_fpn_1x'  # 'cascade_mask_rcnn_r50_fpn_1x'

# Total training epochs.
total_epochs = 20

# Name of the config file.
config_file = MODELS_CONFIG[selected_model]['config_file']

In [ ]:
import os
# config_fname = os.path.join(project_name, 'mmdetection', config_file)
config_fname = '/content/drive/MyDrive/cascade_rcnn_config.py'

assert os.path.isfile(config_fname), '`{}` not exist'.format(config_fname)
config_fname

'/content/drive/MyDrive/cascade_rcnn_config.py'

In [ ]:
import json
annotation_path = '/content/drive/MyDrive/train/annotations.json'
json_file = open(annotation_path)
coco = json.load(json_file)
print(coco["categories"])
classes_names = [category["name"] for category in coco["categories"]]
print(classes_names)

[{'supercategory': 'none', 'id': 1, 'name': 'Ignore'}, {'supercategory': 'none', 'id': 2, 'name': 'Pedestrian'}, {'supercategory': 'none', 'id': 3, 'name': 'People'}, {'supercategory': 'none', 'id': 4, 'name': 'Bicycle'}, {'supercategory': 'none', 'id': 5, 'name': 'Car'}, {'supercategory': 'none', 'id': 6, 'name': 'Van'}, {'supercategory': 'none', 'id': 7, 'name': 'Truck'}, {'supercategory': 'none', 'id': 8, 'name': 'Tricycle'}, {'supercategory': 'none', 'id': 9, 'name': 'Awning-Tricycle'}, {'supercategory': 'none', 'id': 10, 'name': 'Bus'}, {'supercategory': 'none', 'id': 11, 'name': 'Motor'}, {'supercategory': 'none', 'id': 12, 'name': 'Others'}]
['Ignore', 'Pedestrian', 'People', 'Bicycle', 'Car', 'Van', 'Truck', 'Tricycle', 'Awning-Tricycle', 'Bus', 'Motor', 'Others']


In [ ]:
import re
fname = config_fname
with open(fname) as f:
    s = f.read()
    print(s)
    work_dir = re.findall(r"work_dir = \'(.*?)\'", s)[0]
    # Update `num_classes` including `background` class.
    s = re.sub('num_classes=.*?,',
               'num_classes={},'.format(len(classes_names) + 1), s)
    s = re.sub('total_epochs = \d+',
               'total_epochs = {} #'.format(total_epochs), s)
    if "CocoDataset" in s:
        s = re.sub("data_root = 'data/coco/'",
                   "data_root = 'data/'", s)
        s = re.sub("annotations/instances_train2017.json",
                   "trainval.json", s)
        s = re.sub("annotations/instances_val2017.json",
                   "trainval.json", s)
        s = re.sub("annotations/instances_val2017.json",
                   "trainval.json", s)
        s = re.sub("train2017", "images", s)
        s = re.sub("val2017", "images", s)
    else:
        s = re.sub('img_prefix=.*?\],',
                   "img_prefix=data_root + 'images',".format(total_epochs), s)
with open(fname, 'w') as f:
    f.write(s)
!cat {config_fname}

# model settings
model = dict(
    type='CascadeRCNN',
    num_stages=3,
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_scales=[8],
        anchor_ratios=[0.5, 1.0, 2.0],
        anchor_strides=[4, 8, 16, 32, 64],
        target_means=[.0, .0, .0, .0],
        target_stds=[1.0, 1.0, 1.0, 1.0],
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='SmoothL1Loss', beta=1.0 / 9.0, loss_weight=1.0)),
    bbox_roi_extractor=dict(
        type='SingleRoIExtractor',
        roi_layer=dict(type='RoIAlign', out_size=7, sample_num=2),
  

In [ ]:
# !pip uninstall mmcv
# !pip install mmcv-full

In [ ]:
!pip uninstall torch torchvision
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q mmcv terminaltables
!git clone --branch v1.2.0 'https://github.com/open-mmlab/mmdetection.git'
%cd "mmdetection" 
!pip install -r "/content/mmdetection_instance_segmentation_demo/mmdetection/requirements/optional.txt"
!python setup.py install 
!python setup.py develop 
!pip install -r {"requirements.txt"}
!pip install pillow==6.2.1

Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.7/dist-packages/caffe2/*
    /usr/local/lib/python3.7/dist-packages/torch-1.9.0+cu102.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.9.0+cu102
Found existing installation: torchvision 0.10.0+cu102
Uninstalling torchvision-0.10.0+cu102:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torchvision-0.10.0+cu102.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libcudart.459720b2.so.10.2
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libnvjpeg.a6b52b54.so.10
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libpng16.7f72a3c5.so.16
    /usr/local/lib/python3.7/dist-packages/torchvision.

In [ ]:
%cd {project_name}
!python mmdetection/tools/train.py {config_fname}

/content/mmdetection_instance_segmentation_demo
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Traceback (most recent call last):
  File "mmdetection/tools/train.py", line 16, in <module>
    from mmdet.apis import set_random_seed, train_detector
  File "/content/mmdetection_instance_segmentation_demo/mmdetection/mmdet/apis/__init__.py", line 2, in <module>
    from .inference import (async_inference_detector, inference_detector,
  File "/content/mmdetection_instance_segmentation_demo/mmdetection/mmdet/apis/inference.py", line 7, in <module>
    from mmcv.ops import RoIPool
  File "/usr/local/lib/python3.7/dist-packages/mmcv/ops/__init__.py", line 2, in <module>
    from .bbox import bbox_overlaps
  File "/usr/local/lib/python3.7/dist-packages/mmcv/ops/bbox.py", line 4, in <module>
    ext_module = ext_loader.load_ext('_ext', ['bbox_overlaps'])
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/ext_loader.py", line 13, in load_ext
    ext = importlib.import_module('